In [1]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np


In [2]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)

In [3]:
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
# df = df.sample(frac=0.01, random_state=1)
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

In [5]:
# We have to transform categorical variables to use sklearn models
X_processed = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_processed.columns)

In [6]:
X_processed.shape

(32561, 108)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.20, random_state=42)

In [8]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [9]:
from interpret import show
from interpret.perf import ROC

blackbox_perf = ROC(rf.predict_proba).explain_perf(X_test, y_test, name='Blackbox')
show(blackbox_perf)

<!-- http://127.0.0.1:7123/112196598800/ -->

In [10]:
from interpret.blackbox import LimeTabular
from interpret import show

#Blackbox explainers need a predict function, and optionally a dataset
lime = LimeTabular(predict_fn=rf.predict_proba, data=X_train, random_state=1)

#Pick the instances to explain, optionally pass in labels if you have them
lime_local = lime.explain_local(X_test[:5], y_test[:5], name='LIME')

show(lime_local)

<!-- http://127.0.0.1:7123/112273761104/ -->

In [18]:
from interpret.blackbox import ShapKernel

background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=rf.predict_proba, data=background_val, feature_names=feature_names)
shap_local = shap.explain_local(X_test[:5], y_test[:5], name='SHAP')
show(shap_local)


/Users/mayursand/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/Users/mayursand/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/Users/mayursand/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/Users/mayursand/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!



/Users/mayursand/opt/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!



<!-- http://127.0.0.1:7123/112312049040/ -->

In [12]:
background_val = np.median(X_train, axis=0).reshape(1,-1)
background = pd.DataFrame(background_val)
background

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
0,37.0,178263.5,10.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=rf.predict_proba, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Global Sensitivity")

show(sensitivity_global)

<!-- http://127.0.0.1:7123/112273770512/ -->

In [14]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=rf.predict_proba, data=X_train)
pdp_global = pdp.explain_global(name='Partial Dependence')

show(pdp_global)

<!-- http://127.0.0.1:7123/112342909648/ -->

In [15]:
show([blackbox_perf, lime_local,shap_local, sensitivity_global, pdp_global])

<!-- http://127.0.0.1:7123/112382209392/ -->
 Open in new window